# Part IX — Deployment and Production Operations  
## 39. Caching, CDN, and Performance in Production (Redis, Cache Headers, ETags, Safe Invalidation)

In Chapter 20 you learned performance fundamentals and basic caching patterns. In
production, caching becomes a *system*:

- a real cache backend (Redis/Memcached)
- HTTP cache headers (browser + CDN)
- ETags / conditional requests
- CDN strategy for static assets
- safe cache invalidation patterns
- avoiding “cache leaks” (serving user-specific pages to the wrong user)

This chapter gives you a production-grade caching and CDN playbook.

---

## 39.0 Learning Outcomes

By the end you should be able to:

1. Choose the right cache layer for each need:
   - browser cache
   - CDN cache
   - server-side cache (Redis)
2. Configure Redis caching in Django correctly.
3. Use per-view caching safely (only for public content or varied properly).
4. Use template fragment caching safely.
5. Use low-level caching with keys/versioning.
6. Set correct cache headers:
   - `Cache-Control`
   - `ETag`
   - `Last-Modified`
7. Use conditional GETs (304 Not Modified) to reduce bandwidth and latency.
8. Deploy static assets with long cache lifetimes (immutable caching).
9. Avoid common caching security bugs (shared caches + auth).
10. Create a cache invalidation strategy that won’t break production.

---

## 39.1 The Cache Layers (Where Caching Happens)

### 39.1.1 Browser cache
- controlled by response headers
- reduces repeat requests from same user
- handles static assets very well

### 39.1.2 CDN / reverse proxy cache
- caches responses closer to users
- offloads backend
- can cache static, and sometimes public HTML and API GETs

### 39.1.3 Server-side cache (Redis/Memcached)
- used by Django code to cache:
  - query results
  - rendered fragments
  - computed expensive values
- also used for:
  - session storage
  - rate limiting counters
  - channel layers (Channels)
  - Celery broker/result backend

**Important:** Redis can serve multiple roles. In production, you often use separate
Redis databases/instances for:
- cache
- celery
- channels
to reduce blast radius and performance coupling.

---

## 39.2 Configure Redis Cache Backend (Production Standard)

Install Redis client dependency if not present:
- Django’s Redis cache backend often uses `django-redis` (common and feature-rich).

### 39.2.1 Install `django-redis`
```bash
python -m pip install django-redis
python -m pip freeze > requirements.txt
```

### 39.2.2 Configure cache in settings
In `config/settings/prod.py`:

```python
import os

REDIS_CACHE_URL = os.environ.get("REDIS_CACHE_URL", "redis://127.0.0.1:6379/3")

CACHES = {
    "default": {
        "BACKEND": "django_redis.cache.RedisCache",
        "LOCATION": REDIS_CACHE_URL,
        "OPTIONS": {
            "CLIENT_CLASS": "django_redis.client.DefaultClient",
        },
        "TIMEOUT": 300,
    }
}
```

#### Why this is standard
- Redis is fast and shared across workers.
- Cache survives across processes.
- Supports eviction policies and advanced features.

### 39.2.3 Redis cache safety settings (optional but useful)
Add:

```python
"OPTIONS": {
    "CLIENT_CLASS": "django_redis.client.DefaultClient",
    "SOCKET_CONNECT_TIMEOUT": 2,
    "SOCKET_TIMEOUT": 2,
},
```

This prevents hanging if Redis is slow/down.

---

## 39.3 What to Cache (The “High ROI” Targets)

Cache things that are:
- expensive to compute
- requested frequently
- stable for some period
- safe to serve cached to multiple users (public) OR varied appropriately

Common “high ROI” caches:
- article list pages (public)
- top tags sidebar (public)
- sitemap.xml (public)
- expensive DB aggregations (counts)
- computed settings/config objects

Avoid caching:
- per-user private dashboards unless keyed by user
- responses containing CSRF tokens unless varied correctly
- anything with sensitive data in shared caches

---

## 39.4 Per-View Caching in Production (Safe Use)

### 39.4.1 Cache public pages only (default safe rule)
Example: public article list.

```python
from django.views.decorators.cache import cache_page
from django.utils.decorators import method_decorator

@cache_page(60)
def article_list(request):
    ...
```

#### Important: query string variation
Django per-view cache keys include the full path including query string, so:
- `/articles/?tag=django` caches separately from `/articles/?tag=cbv`

### 39.4.2 Vary headers when needed
If a view response varies by header (e.g., language), use `vary_on_headers`.

```python
from django.views.decorators.vary import vary_on_headers

@vary_on_headers("Accept-Language")
@cache_page(300)
def public_page(request):
    ...
```

### 39.4.3 User-specific caching is dangerous if done wrong
If you cache a view that includes `user.username` and don’t vary by session/user,
you can leak user info.

Industry rule:
- don’t cache authenticated HTML pages globally unless you have deep understanding
  (use per-user keys or skip caching)

---

## 39.5 Template Fragment Caching (Often the Best Balance)

Fragment caching avoids caching the whole page, but caches the expensive part.

Example: top tags in article list template:

```django
{% load cache %}
{% cache 300 "top_tags_v1" %}
  ... expensive sidebar ...
{% endcache %}
```

### 39.5.1 Cache key versioning
Include a version string like `"top_tags_v1"`.
When you change the template/logic, bump to `"top_tags_v2"` to avoid stale HTML
format issues.

### 39.5.2 Vary by language or tenant if needed
If your site is multi-tenant or multilingual, add key parts:

```django
{% cache 300 "top_tags_v1" organization.id %}
```

or:

```django
{% cache 300 "top_tags_v1" request.LANGUAGE_CODE %}
```

---

## 39.6 Low-Level Caching (Keyed Values, Not HTML)

Low-level caching is best for:
- computed query results
- API response objects (carefully)
- permission checks caching (carefully)
- expensive external calls (with TTL and fallback)

### 39.6.1 Use stable key patterns
Example:

```python
from django.core.cache import cache

def cache_key(*parts: str) -> str:
    return "myapp:" + ":".join(parts)
```

Then:

```python
key = cache_key("org", str(org.id), "task_counts", "v1")
```

### 39.6.2 Cache invalidation patterns (practical)
- TTL-based: simplest and robust for most UI caches
- Version bump: manual invalidation by key version
- Event-based: delete keys when data changes (signals/services)
  - more complex, can be brittle if you forget to invalidate somewhere

Industry advice:
- Start with TTL + versioning.
- Add event-based invalidation only for very sensitive correctness needs.

---

## 39.7 HTTP Cache Headers (Browser and CDN)

Even without server-side caching, correct HTTP headers reduce load.

### 39.7.1 Cache-Control basics
- `Cache-Control: public, max-age=...` for public resources
- `Cache-Control: private` for user-specific
- `Cache-Control: no-store` for extremely sensitive (e.g., bank statements)
- `Cache-Control: no-cache` means “store but revalidate”

### 39.7.2 Example: sitemap.xml caching
Sitemap changes rarely. Cache it for 1 hour:

```python
from django.views.decorators.cache import cache_page
from django.utils.decorators import method_decorator

# If you have a sitemap view, wrap with cache_page
```

Or set headers manually:

```python
response["Cache-Control"] = "public, max-age=3600"
```

### 39.7.3 ETags and conditional GET (304)
If you support ETags:
- client sends `If-None-Match: <etag>`
- server can return `304 Not Modified` without body

Django provides helpers like `@condition` / `@etag` (from `django.views.decorators.http`).

Example concept:

```python
from django.views.decorators.http import etag

def articles_etag(request):
    # compute a simple etag based on latest updated article timestamp
    latest = Article.objects.published().order_by("-updated_at").values_list("updated_at", flat=True).first()
    return str(latest.timestamp()) if latest else "empty"

@etag(articles_etag)
def article_list(request):
    ...
```

If unchanged, Django returns 304 automatically.

**Industry note:** ETags are powerful but require careful design to avoid heavy DB
work on every request. Often you:
- store a “content version” in cache
- or compute ETag cheaply from a known “last updated” value.

---

## 39.8 CDN for Static Assets (Industry Standard)

Static assets should be cached aggressively:
- hashed filenames (e.g., `site.9a8f3.css`)
- `Cache-Control: public, max-age=31536000, immutable`

Django supports hashed static files using:
- `ManifestStaticFilesStorage` (built-in)
- or WhiteNoise storage backends

### 39.8.1 Enable manifest storage (common)
In production settings:

```python
STORAGES = {
    "staticfiles": {
        "BACKEND": "django.contrib.staticfiles.storage.ManifestStaticFilesStorage",
    },
    "default": {
        "BACKEND": "django.core.files.storage.FileSystemStorage",
    },
}
```

Now collectstatic will:
- produce hashed names for files
- create a manifest mapping original → hashed

Your templates using `{% static 'css/site.css' %}` will automatically resolve to the
hashed name, enabling long caching.

### 39.8.2 Use CDN URL prefix for static
If you use a CDN:

```python
STATIC_URL = "https://cdn.example.com/static/"
```

Then `{% static %}` outputs CDN URLs.

---

## 39.9 Caching and Auth: Avoid the “Private Data Leak” Incident

The most dangerous caching bug:
- caching a response that contains user-specific data
- serving it to someone else

### 39.9.1 Rules to avoid leaks
- Do not per-view cache pages behind login unless you vary by user/session.
- Ensure `Cache-Control: private` on authenticated HTML responses.
- Avoid caching responses that include CSRF tokens in shared caches/CDN without
  proper variation.

### 39.9.2 Varying by Cookie is usually a bad idea
If you add `Vary: Cookie`, shared caches/CDNs will treat every user as separate,
which destroys caching benefit. Better:
- don’t cache those pages
- or cache fragments that are public
- or build a separate public caching layer

---

## 39.10 Production Cache Invalidation Strategy (Practical, Safe)

A simple strategy that works for many apps:

- public pages:
  - cache for short TTL (30–300s)
  - optionally purge on publish events (advanced)
- expensive sidebars:
  - cache fragments for 5–15 minutes
- API GET endpoints:
  - cache only if public and stable; otherwise rely on DB + indexes

If you need strong freshness, add event-driven invalidation in services:

Example: after publishing an article:
- delete keys:
  - `top_tags_v1`
  - `sitemap_v1`
  - `articles_list_*` (harder; often you keep TTL and accept small staleness)

In real systems, “delete all list cache variants” is hard. TTL is often best.

---

# 39.11 Hands-On Lab (Production Caching Upgrade)

## Lab A — Redis cache in dev/staging
1. Install `django-redis`
2. Run Redis
3. Configure `CACHES` to use Redis
4. Verify caching works by:
   - caching a fragment and observing it persists across server reloads (within TTL)

## Lab B — Cache the sitemap
1. Cache sitemap view for 1 hour
2. Add test that second request hits cache (optional; can be tricky)

## Lab C — Static hashing and CDN readiness
1. Enable ManifestStaticFilesStorage
2. Run `collectstatic`
3. Verify static URLs in rendered HTML use hashed filenames
4. Ensure Nginx/CDN can cache static assets for 1 year

---

## 39.12 Exercises (Do These Before Proceeding)

1. Add Redis cache and implement low-level caching for “top tags” query in Python.
2. Add fragment caching for top tags in templates and include a version key.
3. Add per-view caching to one public endpoint and confirm it doesn’t cache
   authenticated state.
4. Implement an ETag for the articles list based on last published update, and
   verify a 304 response when unchanged.
5. Write a “cache policy doc” for your app:
   - what is cached
   - TTL
   - invalidation strategy
   - what must never be cached

---

## 39.13 Chapter Summary

- Production caching is multi-layer: browser + CDN + Redis.
- Use Redis via `django-redis` for shared server-side caching.
- Cache public content and expensive fragments; avoid caching user-specific pages globally.
- Use correct HTTP cache headers and hashed static assets for CDN performance.
- Plan invalidation intentionally; TTL + versioning is often the safest starting point.

---

Next chapter: **40. Operations: Backups, Rollbacks, Incident Response**  
We’ll create real operational runbooks for DB and media backups, restore drills,
rollback strategies, and incident response procedures you can use in production.

<div style='width:100%; display:flex; justify-content:space-between; align-items:center; margin: 1em 0;'>
  <a href='38. ci_cd.ipynb' style='font-weight:bold; font-size:1.05em;'>&larr; Previous</a>
  <a href='../TOC.md' style='font-weight:bold; font-size:1.05em; text-align:center;'>Table of Contents</a>
  <a href='40. operations.ipynb' style='font-weight:bold; font-size:1.05em;'>Next &rarr;</a>
</div>
